In [18]:
import time
import threading
from concurrent.futures import Future


In [2]:
def network_request(number):
    time.sleep(1.0)
    return {"success": True, "result": number ** 2}


def fetch_square(number):
    response = network_request(number)
    if response["success"]:
        print("Result is: {}".format(response["result"]))


if __name__ == "__main__":

    start = time.perf_counter()
    fetch_square(2)
    fetch_square(3)
    fetch_square(4)
    end = time.perf_counter()
    print(f"It takes {end - start}s to finish the calls.")


Result is: 4
Result is: 9
Result is: 16
It takes 3.0033128720001514s to finish the calls.


In [4]:
def wait_print(msg): # A blocking function
    time.sleep(1.0)
    print(msg)


wait_print("Hello 1")
wait_print("Hello 2")
wait_print("Hello 3")


Hello 1
Hello 2
Hello 3


In [8]:
def wait_and_print_async(msg):
    def callback():
        print(msg)
    timer = threading.Timer(1.0, callback)
    timer.start()

wait_and_print_async("Hello 1")
wait_and_print_async("Hello 2")
wait_and_print_async("Hello 3")
print("After submission")


After submission


Hello 1
Hello 2
Hello 3


In [9]:
def cube(x):
    def cube_me():
        print(f"cube: {x**3}")
    timer = threading.Timer(1.0, cube_me)
    timer.start()

cube(2)
cube(8)
cube(10)


cube: 8
cube: 512
cube: 1000


In [16]:
def network_request_async(number, on_done):
    def timer_done():
        on_done({"success": True, "result": number ** 2})
    timer = threading.Timer(1.0, timer_done)
    timer.start()
    print("I am in a network")

def fecth_square(number):
    def on_done(response):
        if response["success"]:
            print(f"Result is {response['result']}")
    network_request_async(number, on_done)

def on_done(result):
    print(result)


In [17]:
network_request_async(2, on_done)

network_request_async(3, on_done)

network_request_async(4, on_done)

print("After submission")


I am in a network
I am in a network
I am in a network
After submission


{'success': True, 'result': 4}
{'success': True, 'result': 9}
{'success': True, 'result': 16}


In [15]:
fecth_square(3)
fecth_square(5)
fecth_square(8)
fecth_square(15)
print("submitted")


submitted


Result is 9
Result is 25
Result is 64
Result is 225


Future

In [19]:
fut = Future()
fut


<Future at 0x7f0a1b359690 state=pending>

In [20]:
fut.set_result("Hullo")
fut


<Future at 0x7f0a1b359690 state=finished returned str>

In [21]:
fut.result()


'Hullo'

In [22]:
# adding a callback to a Future instance
fut = Future()
fut.add_done_callback(lambda future: print(future.result(), flush=True))
fut.set_result("Hello!")


Hello!


In [31]:
def network_request_async(number):
    future = Future()
    result = {"success": True, "result": number ** 2}
    timer = threading.Timer(1.0, lambda: future.set_result(result))
    timer.start()

    return future


def fetch_square(number):
    fut = network_request_async(number)

    def on_done_future(fut):
        response = fut.result()
        if response["success"]:
            print("Result is: {}".format(response["result"]))

    fut.add_done_callback(on_done_future)


if __name__ == "__main__":
    ### First try
    fut = network_request_async(2)

    ### Second try
    # fetch_square(2)


In [33]:
fut.result()


{'success': True, 'result': 4}